**FLIGHT ENRICHMENT ASSIGNMENT TASK**
`Objective:´`´The Objective of this project is to find candidate flights that can be or could be enriched into the already existing Flights operating already on the in this case 14.09.2028 . I will have two dataframes, one entitled FlightScheduleNotSearch containing candidate flights on the day and FlightScheduleSearch which contains the to be searched flights . 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import autopep8
import os
import cProfile
from typing import IO

In [2]:
# setting the directory where the files are being saved to
root_dir = "I:\Flight Enrichment Assignment\data"

files = []

for dir_path, dir_names, file_names in os.walk(root_dir):

    for file_name in file_names:
        # extension is specified over here
        if file_name.endswith('.pkl'):

            files.append(os.path.join(dir_path, file_name))
# Provided files are now showcased here .
files

['I:\\Flight Enrichment Assignment\\data\\CGN0322S.pkl',
 'I:\\Flight Enrichment Assignment\\data\\CGN1025S.pkl']

In [3]:
'''
FS stands for the cleaned Flight Schedule gotten from the cleaned python script . 
0 Stands for the First file and 1 stands for the Second File . 
'''
fs = pd.read_pickle(files[1])

In [4]:
fs.head(5)

,Airline_Code,Leg Type,Via,Origin-Destination,Start Date,End Date,Days Of operations,Operating Times Per Week,time_aftermidnight,Date Range
0,PC,A,ESB,ESB,2018-08-27,2018-08-27,1000000,1,0,0
1,5P,A,HRG,HRG,2018-05-06,2018-05-13,0000007,1,5,7
2,5P,A,HRG,HRG,2018-05-20,2018-05-27,0000007,1,5,7
3,5P,A,HRG,HRG,2018-06-03,2018-06-24,0000007,1,5,21
4,5P,A,HRG,HRG,2018-07-01,2018-10-14,0000007,1,5,105


In [5]:
def reduce_numberofcolumns(fs: IO[str]) -> IO[str]:
    if fs is None:
        raise ValueError("FS cannot be None.")
    if not isinstance(fs, pd.DataFrame):
        raise TypeError("FS must be a pandas DataFrame.")

    # List of columns to be selected
    columns = [
        "Airline_Code", "Leg Type", "Via", "Origin-Destination","Start Date","End Date",
        'Days Of operations', "Operating Times Per Week", "time_aftermidnight",
        "Date Range"
    ]

    fs = fs[columns]

    return fs

In [6]:
'''
This section represents a user input space where one selects a search date . 
'''
# user input
search_date = pd.to_datetime("2018-09-14")

In [7]:
day_of_week = str(search_date.isoweekday())

print("The day of the week of this search date is ", day_of_week)

The day of the week of this search date is  5


In [8]:
def flightschedule_aroundfd(search_date: datetime, fs: IO[str]) -> IO[str]:
    '''
    flightschedule_aroundfd contains all flights that surround the search date . It ensures that 
    the Start Date is before the search date and End Date is after the search date . When an End Date 
    is less than the search date , it is also eliminated and not considered as per the written line of Code. 
    
    '''
    fs = fs[(fs["Start Date"] <= search_date)
            & (search_date <= fs["End Date"])]

    return fs

In [9]:
def flights_on_searchdate(fs: IO[str]) -> IO[str]:
    '''
    flights_on_searchdate is only representative of already kandidate flights that happen on the search date.
    it returns a new distinctive file entitled FS_onsearchdate . 
    '''
    fs_onsearchdate = fs[fs['Days Of operations'].str.contains(day_of_week)]

    return fs_onsearchdate

In [10]:
'''
This is a by the way procudure used to save flight schedules on the search date. This file is later compared to the output files . 
'''
fs_onsearchdate = fs[fs['Days Of operations'].str.contains(day_of_week)]
%store fs_onsearchdate

Stored 'fs_onsearchdate' (DataFrame)


In [11]:
def flights_noton_searchdate(fs: IO[str]) -> IO[str]:
    '''
    flights_noton_searchdate represents those flights that do not fall in the corresponding **Day of the week**. 
    In this case, those are the flights that do not happen on a Friday i.e where the days of operations 
    is 5 are not included in the `´FlightScheduleSearch Data Set`´. This contain the could be candidate flights . 
    '''
    fs_noton_searchdate = fs[~fs['Days Of operations'].str.contains(day_of_week
                                                                    )]

    return fs_noton_searchdate

In [12]:
def mintimedifference(fs_onsearchdate: IO[str],
                      fs_noton_searchdate: IO[str]) -> IO[str]:

    # Enabling copy-on-write mode for pandas .
    pd.options.mode.copy_on_write = True
    '''
    Compares flights on the search date with flights sorrounding the search date that have the same Airline Code, Leg Type,
    Origin-Destination, and Via. It computes the minimum absolute difference in 'time_aftermidnight' 
    between each flight in FS_search and all matching flights of flights not on the search date, then adds this minimum 
    difference as a new column 'time_difference_min' to Flight Schedule not on search date.
    '''

    closest_time_differences = []

    # Iterating through the flight schedule data base for plausible kandidate flight schedules .
    for index, search_row in fs_noton_searchdate.iterrows():

        # Filter FS_notsearch to find rows matching the current FS_search row based on specified criteria
        matching_rows = fs_onsearchdate[
            (fs_onsearchdate['Airline_Code'] == search_row['Airline_Code'])
            & (fs_onsearchdate['Leg Type'] == search_row['Leg Type']) &
            (fs_onsearchdate['Origin-Destination']
             == search_row['Origin-Destination']) &
            (fs_onsearchdate['Via'] == search_row['Via'])]

        # Calculate the minimum absolute time difference where the row match for Airline_Code , leg Type , Destination and Via
        if not matching_rows.empty:

            # Calculate differences between 'time_aftermidnight' of the search row and all matching rows
            time_differences = matching_rows[
                'time_aftermidnight'] - search_row['time_aftermidnight']

            # Calculating the absolute differences
            absolute_time_differences = time_differences.abs()

            # Find the closest (minimum) time difference
            closest_difference = absolute_time_differences.min()
        else:
            closest_difference = pd.NA  # Non matching rows are labeled as NA under the time_difference_min

        # Append the closest difference to the list
        closest_time_differences.append(closest_difference)

    # Add the calculated closest time differences as a new column to FS_search
    fs_noton_searchdate.loc[:,
                            'time_difference_min'] = closest_time_differences.copy(
                            )

    # remove all NA values contained in the time_difference_min since they do not match up .
    fs_noton_searchdate = fs_noton_searchdate[
        fs_noton_searchdate['time_difference_min'].notna()].copy()

    pd.options.mode.copy_on_write = False

    return fs_noton_searchdate

In [13]:
def assign_weight_on_mintime(time_difference_min: int) -> int:
    '''
    The function entitled **assign_weight_on_mintime** is being used here to bin the `time_difference_min` 
    and weight will be allotted to every row based on the minimum time difference in minutes.
    '''
    if time_difference_min <= 10:
        return time_difference_min / 100

    elif time_difference_min <= 90:
        return ((time_difference_min // 10) / 10)
    else:
        return 1
    '''
     The function above performs integer division by 10 , giving out the number of tens in the time 
     difference , the result is later devided by 10 to give us the weight . The number of 10s in 45 is 4
     this is later devided by 10 to allocate a weight of 0.4 
    '''

In [14]:
def assign_weight_on_operationfrequeny(Operating_times: int) -> int:
    '''
     This function assigns weights based on the number of operations with 0.1 representing one weekly 
     operation to 1 indicating 7 times operation . 
    '''
    if Operating_times == 1:
        return 0.1
    elif Operating_times == 2:
        return 0.2
    elif Operating_times == 3:
        return 0.3
    elif Operating_times == 4:
        return 0.5
    elif Operating_times == 5:
        return 0.7
    elif Operating_times == 6:
        return 0.8
    elif Operating_times == 7:
        return 1

In [15]:
def assign_weight_on_daterange(Daterange: datetime) -> int:
    '''
    Date range represents the `´length of time`´ or the difference between the start and end date . 
    The idea is that flights offering longer date ranges are more attractive during `´Flight schedule planning`´ 
    as compared to those with a **shorter course.** 
    '''
    if Daterange >= bins[0] and Daterange <= bins[1]:
        return 0.1
    elif Daterange > bins[1] and Daterange <= bins[2]:
        return 0.2
    elif Daterange > bins[2] and Daterange <= bins[3]:
        return 0.3
    elif Daterange > bins[3] and Daterange <= bins[4]:
        return 0.4
    elif Daterange > bins[4] and Daterange <= bins[5]:
        return 0.5
    elif Daterange > bins[5] and Daterange <= bins[6]:
        return 0.6
    elif Daterange > bins[6] and Daterange <= bins[7]:
        return 0.7
    elif Daterange > bins[7] and Daterange <= bins[8]:
        return 0.8
    elif Daterange >= bins[9] and Daterange <= bins[10]:
        return 0.9
    else:
        return 1


min_value = fs["Date Range"].min()
max_value = fs["Date Range"].max()

bins = np.linspace(
    min_value, max_value,
    11)  # For that matter, it stopps at 11 to produce the 10 bins .
#print(bins)

In [16]:
def apply_penalty_on_weekends(DaysOfOperations: str) -> int:
    '''
    This function applies a penalty of -2 to the week_weight rows if the corresponding Days Of operations 
    contains a 6 or 7 only. This is because flights operating over the weekend do not necessarily operate 
    over the weekend.
    '''
    # Check that '6' and/or '7' are the only non-'0' characters present
    if (DaysOfOperations.count('6') > 0 or DaysOfOperations.count('7') > 0
        ) and DaysOfOperations.count('0') == len(DaysOfOperations) - (
            DaysOfOperations.count('6') + DaysOfOperations.count('7')):

        return -2

    else:

        return 1

In [17]:
def binning_time_weight(fs: IO[str]) -> IO[str]:

    fs['time_bin'] = pd.cut(
        fs['time_weight'],
        bins=[
            -float('inf'), 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0
        ],
        labels=[
            "Exactly 0", "0 - 0.1", "0.1 - 0.2", "0.2 - 0.3", "0.3 - 0.4",
            "0.4 - 0.5", "0.5 - 0.6", "0.6 - 0.7", "0.7 - 0.8", "0.8 - 0.9",
            "0.9 - 1.0"
        ],
        right=True,  #The right edge inclusive
        include_lowest=True  # Include the lowest edge
    )
    return fs

In [18]:
def highlight_cols(fs: IO[str]):
    if fs == 0:
        color = 'red'
    elif fs > 0:
        color = 'blue'
    else:
        'green'
    return 'background-color: %s' % color

In [19]:
def kandidate_flightschedules(fs):

    # Apply the flightschedule function to FS
    fs = flightschedule_aroundfd(search_date, fs)
    print("After flightschedule:", fs is None)

    # Apply the FlightScheduleNotSearch function to the result of flightschedule
    fs_notsearch = flights_on_searchdate(fs)
    print("After FlightScheduleNotSearch:", fs is None)

    # Apply the FlightScheduleSearch function to the result of flightschedule
    fs_search = flights_noton_searchdate(fs)
    print("After FlightScheduleSearch:", fs is None)

    # Apply the mintimedifference function to FS_search
    fs = mintimedifference(fs_notsearch, fs_search)
    print("After mintimedifference:", fs is None)

    # Apply the assign_weight_assignment function to the FS.
    fs["time_weight"] = fs["time_difference_min"].apply(
        assign_weight_on_mintime)
    print("After time_weight assignment", fs is None)

    # Apply the assign_weight_on_operationfrequeny function to the FS .
    fs["week_weight"] = fs["Operating Times Per Week"].apply(
        assign_weight_on_operationfrequeny)
    print("After weight_week_assignment assignment", fs is None)

    # Apply the assign_weight_on_daterange function to the FS .
    fs["Daterange_weight"] = fs["Date Range"].apply(assign_weight_on_daterange)
    print("After weight_Daterange_assignment ", fs is None)

    # Apply the flightschedule function to FS
    reduce_numberofcolumns(fs)
    print("After newfs:", fs is None)

    # in order to avoid key errors, reset the data set .
    fs.reset_index(drop=True, inplace=True)

    # Apply the apply_penalty_on_weekends function to the FS .
    fs["week_weight"] = fs["Days Of operations"].apply(
        apply_penalty_on_weekends)  # Apply the penalty here
    print("After penalty ", fs is None)

    # Apply the eliminate_0_min_difference function to the FS .
    #FS = eliminate_0_min_difference(FS)
    #print("elimianted flights with a 0 difference:", FS is None)

    # Apply the binning_time_weight function to the FS .
    fs = binning_time_weight(fs)
    print("binning_time_weight function applied here :", fs is None)
    
     # color code the time_difference_min rows upon statisfying a specific conditon .
    #fs = display(fs.style.applymap(highlight_cols, 
     # subset = pd.IndexSlice[:, ['time_difference_min']])) 

    return fs

In [20]:
if __name__ == "__main__":
    fs = kandidate_flightschedules(fs)

After flightschedule: False
After FlightScheduleNotSearch: False
After FlightScheduleSearch: False
After mintimedifference: False
After time_weight assignment False
After weight_week_assignment assignment False
After weight_Daterange_assignment  False
After newfs: False
After penalty  False
binning_time_weight function applied here : False


In [21]:
fs_timebin_group = fs.groupby("time_bin").agg({
    "time_difference_min": "mean",
    "time_bin": "count"
}).rename(
    columns={
        "time_bin": "number_flights",
        "time_difference_min": "minimum_time_difference_average"
    }).reset_index()

fs_timebin_group[
    "time_hour"] = fs_timebin_group["minimum_time_difference_average"] / 60
fs_timebin_group["number_flights_percentage"] = (
    fs_timebin_group["number_flights"] /
    fs_timebin_group["number_flights"].sum()) * 100

In [22]:
fs_timebin_group

,time_bin,minimum_time_difference_average,number_flights,time_hour,number_flights_percentage
0,Exactly 0,0.000000,247,0.000000,36.111111
1,0 - 0.1,9.098361,122,0.151639,17.836257
2,0.1 - 0.2,21.938776,49,0.365646,7.163743
3,0.2 - 0.3,32.058824,34,0.534314,4.970760
4,0.3 - 0.4,42.173913,23,0.702899,3.362573
5,0.4 - 0.5,51.470588,17,0.857843,2.485380
6,0.5 - 0.6,62.826087,23,1.047101,3.362573
7,0.6 - 0.7,72.857143,14,1.214286,2.046784
8,0.7 - 0.8,83.333333,6,1.388889,0.877193
9,0.8 - 0.9,90.000000,3,1.500000,0.438596


In [23]:
%store fs 

Stored 'fs' (DataFrame)


In [24]:
cProfile.run('kandidate_flightschedules(fs)')

After flightschedule: False
After FlightScheduleNotSearch: False
After FlightScheduleSearch: False
After mintimedifference: False
After time_weight assignment False
After weight_week_assignment assignment False
After weight_Daterange_assignment  False
After newfs: False
After penalty  False
binning_time_weight function applied here : False
         1748459 function calls (1707062 primitive calls) in 2.290 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.003    0.003 1000719520.py:1(reduce_numberofcolumns)
        1    0.000    0.000    0.006    0.006 1579745615.py:1(flightschedule_aroundfd)
        1    0.000    0.000    0.003    0.003 1982254634.py:1(binning_time_weight)
        1    0.000    0.000    2.290    2.290 3104110032.py:1(kandidate_flightschedules)
        1    0.000    0.000    0.005    0.005 3314665351.py:1(flights_noton_searchdate)
        1    0.034    0.034    2.264    2.264 4

In [25]:
fs

,Airline_Code,Leg Type,Via,Origin-Destination,Start Date,End Date,Days Of operations,Operating Times Per Week,time_aftermidnight,Date Range,time_difference_min,time_weight,week_weight,Daterange_weight,time_bin
0,PC,D,AYT,AYT,2018-06-13,2018-10-10,0030000,1,10,119,1365,1.00,1,0.6,0.9 - 1.0
1,EW,A,PMI,PMI,2018-08-26,2018-09-16,0000007,1,20,21,500,1.00,-2,0.1,0.9 - 1.0
2,XQ,A,AYT,AYT,2018-05-07,2018-10-22,1000000,1,35,168,5,0.05,1,0.8,0 - 0.1
3,XQ,A,AYT,AYT,2018-07-25,2018-09-26,0030000,1,45,63,5,0.05,1,0.3,0 - 0.1
4,8Q,D,IST,IST,2018-08-11,2018-09-22,0000060,1,45,42,0,0.00,-2,0.2,Exactly 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,X3,A,HER,HER,2018-06-10,2018-10-21,0000007,1,1425,133,155,1.00,-2,0.7,0.9 - 1.0
680,8Q,A,IST,IST,2018-09-09,2018-09-30,0000007,1,1435,21,0,0.00,-2,0.1,Exactly 0
681,8Q,A,IST,IST,2018-09-10,2018-09-24,1000000,1,1435,14,0,0.00,1,0.1,Exactly 0
682,8Q,A,IST,IST,2018-09-11,2018-09-25,0200000,1,1435,14,0,0.00,1,0.1,Exactly 0
